# Feed-forward сети

Итак, давайте потренируемся тренировать нейронные сети прямого распространения (так как делали на паре)
При этом попробуем создать свою функцию активации на одном из слоев

In [1]:
# Сделаем необходимые импорты
# import torch
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import optim
# import torchvision
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms

In [2]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = CIFAR100(root='data/',
                         train=True,  
                         transform=transforms.ToTensor(), 
                         download=True)

Files already downloaded and verified


In [3]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=64, 
                          shuffle=True)

In [4]:
test_dataset = CIFAR100(root='./data', train=False,
                        download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=4,
                         shuffle=False)

Files already downloaded and verified


In [24]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации
class FFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * hidden_dim)
#         self.fc2 = nn.Linear(16 * hidden_dim, 8 * hidden_dim)
#         self.fc3 = nn.Linear(8 * hidden_dim, 4 * hidden_dim)
        self.fc4 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.fc5 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc6 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.1)
#         x = self.fc2(x)
#         x = F.leaky_relu(x, 0.1)
#         x = self.fc3(x)
#         x = F.leaky_relu(x, 0.1)
        x = self.fc4(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc5(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc6(x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.1)
#         x = self.fc2(x)
#         x = F.leaky_relu(x, 0.1)
#         x = self.fc3(x)
#         x = F.leaky_relu(x, 0.1)
        x = self.fc4(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc5(x)
        x = F.leaky_relu(x, 0.1)
        x = self.fc6(x)
        x = F.softmax(x)
        return x

In [27]:
ff_net = FFNet(3072, 200, len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ff_net.parameters(), lr=0.001, momentum=0.9)

In [22]:
# Запустить обучение (по аналогии с тем, что делали на паре)
from tqdm import tqdm

In [28]:
for epoch in tqdm(range(20)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = ff_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
         
        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

[1,     1] loss: 0.002
[1,   301] loss: 0.691
[1,   601] loss: 0.690


  5%|████▏                                                                              | 1/20 [00:14<04:33, 14.41s/it]

[2,     1] loss: 0.002
[2,   301] loss: 0.689
[2,   601] loss: 0.687


 10%|████████▎                                                                          | 2/20 [00:29<04:20, 14.48s/it]

...

[19,     1] loss: 0.002
[19,   301] loss: 0.525
[19,   601] loss: 0.521


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:34<00:14, 14.35s/it]

[20,     1] loss: 0.002
[20,   301] loss: 0.520
[20,   601] loss: 0.518


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:48<00:00, 14.44s/it]

Training is finished!
